In [315]:
import numpy as np
import pandas as pd
import gurobipy as gp
from gurobipy import GRB
from scipy.spatial import distance_matrix

In [316]:
outlets, coor_x, coor_y = gp.multidict(
    {
        "outlet1": [9.4888, 5.6817],
        "outlet2": [8.7928, 10.3868],
        "outlet3": [11.5960, 3.9294],
        "outlet4": [11.5643, 4.4325],
        "outlet5": [5.6756, 9.9658],
        "outlet6": [9.8497, 17.6632],
        "outlet7": [9.1756, 6.1517],
        "outlet8": [13.1385, 11.8569],
        "outlet9": [15.4663, 8.8721],
        "outlet10": [15.5464, 15.5868],
    }
)

In [317]:
coor = pd.DataFrame(
    [coor_x.values(), coor_y.values()], columns=outlets, index=["x", "y"]
).T
coor

,x,y
outlet1,9.4888,5.6817
outlet2,8.7928,10.3868
outlet3,11.5960,3.9294
outlet4,11.5643,4.4325
outlet5,5.6756,9.9658
outlet6,9.8497,17.6632
outlet7,9.1756,6.1517
outlet8,13.1385,11.8569
outlet9,15.4663,8.8721
outlet10,15.5464,15.5868


In [318]:
dist = pd.DataFrame(distance_matrix(coor, coor))
dist

,0,1,2,3,4,5,6,7,8,9
0,0.000000,4.756299,2.740592,2.422437,5.735330,11.986934,0.564796,7.173103,6.775630,11.610578
1,4.756299,0.000000,7.039598,6.567717,3.145501,7.352757,4.252365,4.587625,6.843239,8.523562
2,2.740592,7.039598,0.000000,0.504098,8.455132,13.844379,3.285872,8.076173,6.277699,12.308559
3,2.422437,6.567717,0.504098,0.000000,8.080482,13.341337,2.943049,7.589455,5.910639,11.843797
4,5.735330,3.145501,8.455132,8.080482,0.000000,8.756316,5.176617,7.698775,9.851598,11.359064
5,11.986934,7.352757,13.844379,13.341337,8.756316,0.000000,11.531220,6.673030,10.432144,6.063318
6,0.564796,4.252365,3.285872,2.943049,5.176617,11.531220,0.000000,6.946502,6.853720,11.384560
7,7.173103,4.587625,8.076173,7.589455,7.698775,6.673030,6.946502,0.000000,3.785193,4.439610
8,6.775630,6.843239,6.277699,5.910639,9.851598,10.432144,6.853720,3.785193,0.000000,6.715178
9,11.610578,8.523562,12.308559,11.843797,11.359064,6.063318,11.384560,4.439610,6.715178,0.000000


In [319]:
m = gp.Model()

In [320]:
x = m.addVars(10, name="set_station", vtype=GRB.BINARY)

In [321]:
# y:is_covered[i,j] 第j个网点被第i个网点的供应站覆盖
y = m.addMVar((10, 10), name="is_covered", vtype=GRB.BINARY)

In [322]:
m.setObjective(x.sum())

In [323]:
m.addConstrs((y[i, :].sum() >= 1 for i in range(10)), name="要求被覆盖限制")

{0: <MConstr ()>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>,
 5: <MConstr () *awaiting model update*>,
 6: <MConstr () *awaiting model update*>,
 7: <MConstr () *awaiting model update*>,
 8: <MConstr () *awaiting model update*>,
 9: <MConstr () *awaiting model update*>}

In [324]:
m.addConstrs((y[:, j].sum() <= 5 for j in range(10)), name="最大供应限制")

{0: <MConstr () *awaiting model update*>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>,
 5: <MConstr () *awaiting model update*>,
 6: <MConstr () *awaiting model update*>,
 7: <MConstr () *awaiting model update*>,
 8: <MConstr () *awaiting model update*>,
 9: <MConstr () *awaiting model update*>}

In [325]:
m.addConstrs(
    (dist.iloc[i, j] * y[i, j] <= 10 * x[i] for j in range(10) for i in range(10)),
    name="距离限制",
)

{(0, 0): <MConstr () *awaiting model update*>,
 (0, 1): <MConstr () *awaiting model update*>,
 (0, 2): <MConstr () *awaiting model update*>,
 (0, 3): <MConstr () *awaiting model update*>,
 (0, 4): <MConstr () *awaiting model update*>,
 (0, 5): <MConstr () *awaiting model update*>,
 (0, 6): <MConstr () *awaiting model update*>,
 (0, 7): <MConstr () *awaiting model update*>,
 (0, 8): <MConstr () *awaiting model update*>,
 (0, 9): <MConstr () *awaiting model update*>,
 (1, 0): <MConstr () *awaiting model update*>,
 (1, 1): <MConstr () *awaiting model update*>,
 (1, 2): <MConstr () *awaiting model update*>,
 (1, 3): <MConstr () *awaiting model update*>,
 (1, 4): <MConstr () *awaiting model update*>,
 (1, 5): <MConstr () *awaiting model update*>,
 (1, 6): <MConstr () *awaiting model update*>,
 (1, 7): <MConstr () *awaiting model update*>,
 (1, 8): <MConstr () *awaiting model update*>,
 (1, 9): <MConstr () *awaiting model update*>,
 (2, 0): <MConstr () *awaiting model update*>,
 (2, 1): <MCo

In [326]:
m.addConstrs((x[i] == y[i][i] for i in range(10)), name="本网点建供应站")

{0: <MConstr () *awaiting model update*>,
 1: <MConstr () *awaiting model update*>,
 2: <MConstr () *awaiting model update*>,
 3: <MConstr () *awaiting model update*>,
 4: <MConstr () *awaiting model update*>,
 5: <MConstr () *awaiting model update*>,
 6: <MConstr () *awaiting model update*>,
 7: <MConstr () *awaiting model update*>,
 8: <MConstr () *awaiting model update*>,
 9: <MConstr () *awaiting model update*>}

In [327]:
m.addConstrs(
    (x[i] >= y[i][j] for i in range(10) for j in range(10)),
    name="本网点不建站则其他不会被本网点覆盖",
)

{(0, 0): <MConstr () *awaiting model update*>,
 (0, 1): <MConstr () *awaiting model update*>,
 (0, 2): <MConstr () *awaiting model update*>,
 (0, 3): <MConstr () *awaiting model update*>,
 (0, 4): <MConstr () *awaiting model update*>,
 (0, 5): <MConstr () *awaiting model update*>,
 (0, 6): <MConstr () *awaiting model update*>,
 (0, 7): <MConstr () *awaiting model update*>,
 (0, 8): <MConstr () *awaiting model update*>,
 (0, 9): <MConstr () *awaiting model update*>,
 (1, 0): <MConstr () *awaiting model update*>,
 (1, 1): <MConstr () *awaiting model update*>,
 (1, 2): <MConstr () *awaiting model update*>,
 (1, 3): <MConstr () *awaiting model update*>,
 (1, 4): <MConstr () *awaiting model update*>,
 (1, 5): <MConstr () *awaiting model update*>,
 (1, 6): <MConstr () *awaiting model update*>,
 (1, 7): <MConstr () *awaiting model update*>,
 (1, 8): <MConstr () *awaiting model update*>,
 (1, 9): <MConstr () *awaiting model update*>,
 (2, 0): <MConstr () *awaiting model update*>,
 (2, 1): <MCo

In [328]:
m.write("IP.lp")

In [329]:
m.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.4.0 23E224)

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 230 rows, 110 columns and 610 nonzeros
Model fingerprint: 0x2676a516
Variable types: 0 continuous, 110 integer (110 binary)
Coefficient statistics:
  Matrix range     [5e-01, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 10.0000000
Presolve removed 230 rows and 110 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 10 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.000000000000e+01, best bound 1.000000000000e+01, gap 0.0000%


In [330]:
print(f"Objective value: {m.ObjVal}")
print("Optimal solution:")
for i in m.getVars():
    if i.X > 1e-6:
        print(f"{i.VarName} = {i.X}", end="\n")

Objective value: 10.0
Optimal solution:
set_station[0] = 1.0
set_station[1] = 1.0
set_station[2] = 1.0
set_station[3] = 1.0
set_station[4] = 1.0
set_station[5] = 1.0
set_station[6] = 1.0
set_station[7] = 1.0
set_station[8] = 1.0
set_station[9] = 1.0
is_covered[0,0] = 1.0
is_covered[0,6] = 1.0
is_covered[1,1] = 1.0
is_covered[2,2] = 1.0
is_covered[2,3] = 1.0
is_covered[3,2] = 1.0
is_covered[3,3] = 1.0
is_covered[4,4] = 1.0
is_covered[5,5] = 1.0
is_covered[6,0] = 1.0
is_covered[6,6] = 1.0
is_covered[7,7] = 1.0
is_covered[8,8] = 1.0
is_covered[9,9] = 1.0
